<a href="https://colab.research.google.com/github/ljpetkovic/corr_OCR_NER/blob/main/extraire_EN_xml_csv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Préliminaires :
* Télécharger / cloner le dépôt GitHub https://github.com/ljpetkovic/corr_OCR_NER 

## 1&nbsp;&nbsp;&nbsp;Extraire du texte brut à partir de la balise `<body>`

In [ ]:
import os
from os import listdir
from os.path import isfile, join
from bs4 import BeautifulSoup

test_path = './test/'
txt_path = './txt_non_corr/'
 

    
with open(txt_path+'txt_non_corr_3.txt', 'w') as txt:


    for filename in os.listdir(test_path)[:2]:
        f = os.path.join(test_path,filename)

        with open(f, 'r') as f:


            # 'xml' is the parser used. For html files, which BeautifulSoup is typically used for, it would be 'html.parser'.
            soup = BeautifulSoup(f, 'xml')
            names = soup.find('body')
            n = names.text
            print(n)
            txt.write(n)











Poesies
DE


Valentin
(HENRI BOURETTE
PUBLIEES PAR

MM. T. LAROUSSILHE ET J. COMBARIEU


A. LEMERRE

LIBRAIRE-EDITEUR
7-31. Passage Choiseul
PARIS

J. GIRMA

LIBRAIRE
24. Boulevard Gambetta
GAHORS
M DCCC LXXXV




Poésies de valentin

JUSTIFICATION DU TIRAGE
Papier vélin teinté. 460 exemplaires
Papier de Hollande (numérotés). 40 —
500 exemplaires


cliché Honore fils CAHORS



Poesies
DE


Valentin
(HENRI BOURETTE) 
PUBLIÉES PAR

F. LAROUSSlLHE ET J. COMBARIEU


A. LEMERRE

I IBRAIRE-EDITEUR
27-37, Passage Choiseul
PARIS

J. GIRMA

LIBRAIRE
24, Boulevard Gambetta
CAHORS
M DCCC LXXXV




Notice biographique
CAHORS n’a pas donné le jour à Valentin (Henri


Bourette). Notre poète était né dans le nord


de la France où son père occupait l’emploi


de vérificateur des Contributions directes. Mais il fut


amené tout enfant dans l’ancienne capitale du Quercy,


evenue ensuite la résidence de ses parents et de son


seul frère, âgé de vingt ans de plus que lui. Il fit ses


classes

## 1a.&nbsp;&nbsp;&nbsp;Retirer les lignes vides après l’extraction du texte

In [ ]:
!ex -s +'v/\S/d' -cwq './txt_non_corr/txt_non_corr_3.txt'

## 2&nbsp;&nbsp;&nbsp;Lancer [`spacy`](https://spacy.io/models/fr) sur le fichier `.txt`

In [ ]:
import spacy, csv, os

path = './txt_non_corr/txt_non_corr_3.txt'

path_of_the_directory_2= './csv_non_corr/' 
 
nlp = spacy.load("fr_core_news_lg") # faire pareil pour fr_core_news_md et fr_core_news_lg
nlp.max_length = 1511212

with open(path, encoding='utf-8', errors='ignore') as file_in:
    with open(path_of_the_directory_2+'EN_non_corr_lg_3.csv', 'w') as doc_csv:

    # create the csv writer
        writer = csv.writer(doc_csv)

        text = file_in.read()
        doc = nlp(text)
        for ent in doc.ents:
            #print(ent, ent.label_)
            writer.writerow([str(ent)])   
            
 


## 3&nbsp;&nbsp;&nbsp;Évaluation avec [`Difflib`](https://docs.python.org/3/library/difflib.html)

### Différences entre les EN avant et après la correction d'OCR
* Insertions (`+`)
* Suppressions (`-`)
* Substitutions (`?`)

In [ ]:
import difflib

ratio = "./changements.txt"

file_non_corr_3 = "./csv_non_corr/EN_non_corr_lg_3.csv"
file_non_corr_6 = "./csv_non_corr/EN_non_corr_lg_6.csv"
file_non_corr_9 = "./csv_non_corr/EN_non_corr_lg_9.csv"

file_corr_3 = "./csv_corr/EN_corr_lg_3.csv"
file_corr_6 = "./csv_corr/EN_corr_lg_6.csv"
file_corr_9 = "./csv_corr/EN_corr_lg_9.csv"

diff = difflib.ndiff(open(file_non_corr_9).readlines(),open(file_corr_9).readlines())
diff_out = ''.join(diff)
print (diff_out)

with open(ratio, 'w') as r:
    r.write(diff_out)

  "Poesies
  DE
  Valentin
  "
  HENRI
  PUBLIEES
- MM. T. LAROUSSILHE
+ "MM. T. LAROUSSILHE"
? +                  +
  "Passage Choiseul
  PARIS
  J. GIRMA
  LIBRAIRE
  24"
  "Boulevard Gambetta
- GAHORS
? ^
+ CAHORS
? ^
  M"
- "DCCC LXXXV
+ DCCC
+ "Poésies de valentin
+ JUSTIFICATION DU TIRAGE
  "
- Papier vélin teinté
+ "Papier vélin teinté"
? +                   +
- Papier de Hollande
+ "Papier de Hollande"
? +                  +
  "Honore fils CAHORS
  Poesies
  DE
  Valentin"
  HENRI
  PUBLIÉES
  "Passage Choiseul
  PARIS
  J. GIRMA
- "
+ LIBRAIRE
+ 24"
  "Boulevard Gambetta
  CAHORS
- M"
- DCCC
+ M DCCC"
? ++    +
+ "Notice biographique
- CAHORS
+ CAHORS"
?       +
  Valentin
  "Henri
- Bourette"
?    ^
+ Boulette"
?    ^
- la France
+ "la France"
? +         +
  Contributions
  Quercy
  Valette
- Lycée de
+ "Lycée de"
? +        +
  Cahors
  "NOTICE BIOGRAPHIQUE
  franchir les limites."
- Réformateur du Lot
+ "Réformateur du Lot"
? +                  +
  "Molière,
  Racine"
- Al

### Sortie HTML (visualisation des diffs)

In [ ]:
from difflib import HtmlDiff

line1 = "./csv_non_corr/EN_non_corr_lg_9.csv"
line2 = "./csv_corr/EN_corr_lg_9.csv"
d = HtmlDiff()
difference = d.make_file(open(line1).readlines(), open(line2).readlines())
with open("diff.html", "w") as f:
    for line in difference.splitlines():
        print (line, file=f)
        
from IPython.display import IFrame

IFrame(src='./diff.html', width=700, height=600)


### Ratio entre les EN issues de différentes portions de texte

Démo

In [ ]:
from difflib import SequenceMatcher
line1 = "aaa bbb"
line2 = "aaa abb"
sm = SequenceMatcher(a=line1, b=line2)
print (sm.ratio())

0.8571428571428571


Texte non corrigé

* 3 vs. 6 fichiers non corrigés <br>

In [ ]:
sm = SequenceMatcher(a=file_non_corr_3, b=file_non_corr_6)
print (sm.ratio())

0.9714285714285714


* 3 vs. 9 fichiers non corrigés <br>

In [ ]:
sm = SequenceMatcher(a=file_non_corr_3, b=file_non_corr_9)
print (sm.ratio())

0.9714285714285714


Texte corrigé

* 3 vs. 6 fichiers corrigés <br>

In [ ]:
sm = SequenceMatcher(a=file_corr_3, b=file_corr_6)
print (sm.ratio())

0.9629629629629629


* 3 vs. 9 fichiers corrigés <br>

In [ ]:
sm = SequenceMatcher(a=file_corr_3, b=file_corr_9)
print (sm.ratio())

0.9629629629629629


#### Pourquoi le même ratio ?

### Ratio entre les EN avant et après la correction d'OCR

* 3 fichiers

In [ ]:
sm = SequenceMatcher(a=file_non_corr_3, b=file_corr_3)
print (sm.ratio())

0.8709677419354839


* 6 fichiers

In [ ]:
sm = SequenceMatcher(a=file_non_corr_6, b=file_corr_6)
print (sm.ratio())

0.8709677419354839


* 9 fichiers

In [ ]:
sm = SequenceMatcher(a=file_non_corr_9, b=file_corr_9)
print (sm.ratio())

0.8709677419354839


#### Pourquoi le même ratio ?

### Récupérer les types de changements des EN

* Insertions
* Suppressions
* Substitutions

Démo

In [ ]:
>>> a = "Paris London"
>>> b = "Prais London"
>>> s = SequenceMatcher(None, a, b)
>>> for tag, i1, i2, j1, j2 in s.get_opcodes():
...     print('{:7}   a[{}:{}] --> b[{}:{}] {!r:>8} --> {!r}'.format(
...         tag, i1, i2, j1, j2, a[i1:i2], b[j1:j2]))

equal     a[0:1] --> b[0:1]      'P' --> 'P'
insert    a[1:1] --> b[1:2]       '' --> 'r'
equal     a[1:2] --> b[2:3]      'a' --> 'a'
delete    a[2:3] --> b[3:3]      'r' --> ''
equal     a[3:12] --> b[3:12] 'is London' --> 'is London'


TGB

In [ ]:
changements = "./changements_types.csv"
with open(changements, 'w') as r:
    
    with open("./csv_non_corr/EN_non_corr_lg_9.csv", "r") as my_file, open("./csv_corr/EN_corr_lg_9.csv", "r") as my_file_2:
        file_1 = my_file.read().splitlines(0)
        file_2 = my_file_2.read().splitlines(0)

        s = SequenceMatcher(None, file_1, file_2)
        r.write('Type de changement\t Fichier non corrigé\t Fichier corrigé\t Mots du fichier non corrigé remplacés dans le fichier corrigé\t Mots remplaçants dans le fichier corrigé\t')
        for tag, i1, i2, j1, j2 in s.get_opcodes():
            results = '{:7}\t   EN_non_corr_lg_9.csv[{}:{}]\t EN_corr_lg_9.csv[{}:{}]\t {!r:>8}\t {!r}'.format('\n'+
                tag, i1, i2, j1, j2, file_1[i1:i2], file_2[j1:j2])
            print(results)
            r.write(results)


equal 	   EN_non_corr_lg_9.csv[0:13]	 EN_corr_lg_9.csv[0:13]	 ['"Poesies', 'DE', 'Valentin', '"', 'HENRI', 'PUBLIEES', 'MM. T. LAROUSSILHE', '"Passage Choiseul', 'PARIS', 'J. GIRMA', 'LIBRAIRE', '24"', '"Boulevard Gambetta']	 ['"Poesies', 'DE', 'Valentin', '"', 'HENRI', 'PUBLIEES', 'MM. T. LAROUSSILHE', '"Passage Choiseul', 'PARIS', 'J. GIRMA', 'LIBRAIRE', '24"', '"Boulevard Gambetta']

replace	   EN_non_corr_lg_9.csv[13:14]	 EN_corr_lg_9.csv[13:14]	 ['GAHORS']	 ['CAHORS']

equal 	   EN_non_corr_lg_9.csv[14:15]	 EN_corr_lg_9.csv[14:15]	   ['M"']	 ['M"']

replace	   EN_non_corr_lg_9.csv[15:16]	 EN_corr_lg_9.csv[15:18]	 ['"DCCC LXXXV']	 ['DCCC', '"Poésies de valentin', 'JUSTIFICATION DU TIRAGE']

equal 	   EN_non_corr_lg_9.csv[16:28]	 EN_corr_lg_9.csv[18:30]	 ['"', 'Papier vélin teinté', 'Papier de Hollande', '"Honore fils CAHORS', 'Poesies', 'DE', 'Valentin"', 'HENRI', 'PUBLIÉES', '"Passage Choiseul', 'PARIS', 'J. GIRMA']	 ['"', 'Papier vélin teinté', 'Papier de Hollande', '"Honore fil

equal 	   EN_non_corr_lg_9.csv[10876:10885]	 EN_corr_lg_9.csv[10355:10364]	 ['Le Torrent et la Rivière', 'La Tortue', 'Canards.', 'X', 'Le Trésor', 'Tribut', 'Alexandre', 'Les Vautours', 'Le Vieillard']	 ['Le Torrent et la Rivière', 'La Tortue', 'Canards.', 'X', 'Le Trésor', 'Tribut', 'Alexandre', 'Les Vautours', 'Le Vieillard']

replace	   EN_non_corr_lg_9.csv[10885:10886]	 EN_corr_lg_9.csv[10364:10365]	  ['Ane']	 ['Une.']

equal 	   EN_non_corr_lg_9.csv[10886:10892]	 EN_corr_lg_9.csv[10365:10371]	 ['Le Vieillard', 'Enfants.', 'Le Vieillard', 'La Vieille', 'Le Villageois', 'Serpent']	 ['Le Vieillard', 'Enfants.', 'Le Vieillard', 'La Vieille', 'Le Villageois', 'Serpent']

replace	   EN_non_corr_lg_9.csv[10892:10893]	 EN_corr_lg_9.csv[10371:10372]	 ['Les Voleurs et l’Ane']	 ['Les Voleurs et l’Une']

equal 	   EN_non_corr_lg_9.csv[10893:10898]	 EN_corr_lg_9.csv[10372:10377]	 ['I', 'Tours', '"À', 'te', 'iiïilaff']	 ['I', 'Tours', '"À', 'te', 'iiïilaff']

replace	   EN_non_corr_lg_9.csv[10

### D'autres tests

In [ ]:
import difflib as dl

# opening the file in read mode
my_file = open("./csv_non_corr/EN_non_corr_lg_3.csv", "r")
my_file_2 = open("./csv_corr/EN_corr_lg_3.csv", "r")

# reading the file
data = my_file.read()
data_2 = my_file_2.read()

# replacing end of line('/n') with ' ' and
# splitting the text it further when '.' is seen.
data_into_list = data.replace('\n', ' ').split(".")
data_into_list_2 = data_2.replace('\n', ' ').split(".")

# printing the data
# print(data_into_list)



for diff in dl.unified_diff(data_into_list, data_into_list_2):
    print(diff)
    
my_file.close()
my_file_2.close()


--- 

+++ 

@@ -1,99 +1,94 @@

-"Poesies DE Valentin " HENRI PUBLIEES MM
+"Poesies DE Valentin " HENRI PUBLIEES "MM
  T
- LAROUSSILHE "Passage Choiseul PARIS J
- GIRMA LIBRAIRE 24" "Boulevard Gambetta GAHORS M" "DCCC LXXXV " Papier vélin teinté Papier de Hollande "Honore fils CAHORS Poesies DE Valentin" HENRI PUBLIÉES "Passage Choiseul PARIS J
- GIRMA " "Boulevard Gambetta CAHORS M" DCCC CAHORS Valentin "Henri Bourette" la France Contributions Quercy Valette Lycée de Cahors "NOTICE BIOGRAPHIQUE franchir les limites
-" Réformateur du Lot "Molière, Racine" Alfred de Musset Lamartine "Barbier, Shakespeare" Milton Dante "Le Tasse, son père" France l’Afrique Valentin NOTICE BIOGRAPHIQUE III Valentin Valentin Algérie "IV NOTICE BIOGRAPHIQUE des inoffensives" Muse Afrique Silo A Marseille de France "Valentin avait" "NOTICE BIOGRAPHIQUE V " Afrique Afrique "Cour d’appel de Saint-Denis" Réunion "Cahors
- Valentin" Bordeaux Cahors "VI NOTICE BIOGRAPHIQUE " M
+ LAROUSSILHE" "Passage Choiseul PARI